In [16]:
from tensorflow.keras.layers import Flatten,Dense,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau
import numpy as np
import util7 as u

In [17]:
input_shape = (320,320,3)
target_size = input_shape[:2]

In [18]:
def Model(drop_out=None):
    vgg19 = VGG19(include_top=False,
                weights='imagenet',
                input_shape=input_shape)

    unfreeze = ['block5_conv1', 'block5_conv2', 'block5_conv3', 'block5_conv4']

    for layer in vgg19.layers:
        if layer.name in unfreeze:
            layer.trainable = True
        else:
            layer.trainable = False

    model = Sequential()
    model.add(vgg19)    # 將 vgg16 做為一層
    model.add(Flatten())

    #model.add(Dropout(drop_out))  # 丟棄法

    model.add(Dense(512, activation='relu'))
    model.add(Dense(512, activation='relu'))

    model.add(Dense(3, activation='softmax'))

    model.compile(optimizer=Adam(learning_rate=1e-4),
            loss='categorical_crossentropy',
            metrics=['acc'])

    return model

In [19]:
def test(model):
    ###生成資料訓練模型
    gobj = ImageDataGenerator(rescale=1./255,
                            rotation_range=180,      #←隨機旋轉 -180~180 度
                            #brightness_range=(0.3, 0.9),
                            zoom_range=0.1,         #←隨機水平或垂直縮放影像 10% (90%~110%)
                            #shear_range=1,         #←隨機順時針傾斜影像 0~1 度
                            horizontal_flip=True, #←隨機水平翻轉影像
                            vertical_flip=True   #←隨機垂直翻轉影像
                            #width_shift_range=0.1,  #←隨機向左或右平移 10% 寬度以內的像素
                            #height_shift_range=0.1, #←隨機向上或下平移 10% 高度以內的像素
                            )

    trn_gen = gobj.flow_from_directory(
      'train',
      target_size=target_size,
      batch_size=140, #total 5600
      class_mode='categorical')

    gobj = ImageDataGenerator(rescale=1./255)

    val_gen = gobj.flow_from_directory( #←建立生成驗證資料的走訪器
      'dev',
      target_size=target_size,
      batch_size=100, #total 800
      class_mode='categorical')

    early_stopping = EarlyStopping(monitor='val_acc', min_delta=0,  patience=15, verbose=1)

    model_checkpoint = ModelCheckpoint(filepath='2_vgg19_weights.{epoch:02d}-{val_acc:.2f}.h5', 
                                    monitor='val_acc',verbose=1, save_best_only=True)

    reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5,
                              verbose=1, cooldown=5,min_lr=1e-10)

    history = model.fit(trn_gen,
                      steps_per_epoch=40,  #←設定每週期從訓練走訪器抽取 40 批次 (=5600/140)
                      epochs=300,
                      validation_data=val_gen, #←指定驗證用的走訪器
                      validation_steps=8, #←設定每週期從驗證走訪器抽取 8 批次 (=800/100)
                      callbacks=[early_stopping,model_checkpoint,reduce_lr])


    u.plot(history.history, ('loss','val_loss'),  # 繪製損失值歷史線圖
          'Training & Validation loss', ('Epochs', 'Loss'))

    u.plot(history.history, ('acc','val_acc'),    # 繪製準確度歷史線圖
          'Training & Validation accuracy',('Epochs', 'Accuracy'))

    return history

In [ ]:
model = Model()
history = test(model)

Found 5600 images belonging to 3 classes.
Found 800 images belonging to 3 classes.
Epoch 1/300
40/40 [==============================] - ETA: 0s - loss: 0.9416 - acc: 0.5420
Epoch 00001: val_acc improved from -inf to 0.59625, saving model to 2_vgg19_weights.01-0.60.h5
40/40 [==============================] - 137s 3s/step - loss: 0.9416 - acc: 0.5420 - val_loss: 0.7803 - val_acc: 0.5962 - lr: 1.0000e-04
Epoch 2/300
40/40 [==============================] - ETA: 0s - loss: 0.6638 - acc: 0.6920
Epoch 00002: val_acc improved from 0.59625 to 0.73250, saving model to 2_vgg19_weights.02-0.73.h5
40/40 [==============================] - 138s 3s/step - loss: 0.6638 - acc: 0.6920 - val_loss: 0.6260 - val_acc: 0.7325 - lr: 1.0000e-04
Epoch 3/300
40/40 [==============================] - ETA: 0s - loss: 0.6054 - acc: 0.7221
Epoch 00003: val_acc did not improve from 0.73250
40/40 [==============================] - 137s 3s/step - loss: 0.6054 - acc: 0.7221 - val_loss: 0.6172 - val_acc: 0.7237 - lr: 1.00

In [ ]:
import tensorflow as tf


In [ ]:
tf.__version__